In [1]:
from pathlib import Path 
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce,api
import shutil
import datetime
from datetime import date


sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm010483",
    security_token="nc20DsgxOHWVR9OHZ78s7TRp",
)

In [2]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
    ]
]
Empresas_SF.shape

(6886, 7)

In [3]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

(6868, 5)

In [4]:
#Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A"
    ]
]

Programas_empresas.shape

(6892, 9)

fin empresas y comienzo de rescate de pacientes


In [5]:
#CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone  from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        'Phone'
        
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
    }
)
Pacientes_SF.shape

(1621917, 9)

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.

In [6]:
#buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
    columns={
        "Id": "Id Pacientes_Programas__c",
        "Paciente__c": "Id Account",
        "Programas_por_Empresas__c": "Programas_por_Empresas__c",
        "padre__c": "Nombre Campaña"        
    }
)
Paciente_Programas_SF.shape

(1184021, 4)

In [7]:
#Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_empresas,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c", 
)
empresas_unidas.shape

(1184021, 12)

In [8]:
Programas_empresas.to_excel("F:/Empresas SF/reportes/empresas.xlsx",index=False)

PermissionError: [Errno 13] Permission denied: 'F:/Empresas SF/reportes/empresas.xlsx'

In [ ]:
empresas_unidas.head()

,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Codigo EMP,Codigo A
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,MetLife,0018c00002axQaeAAE,Serv. De Comerc. Y Ases. Callbusiness SPA,DBCH - CNSTD 2019,MetLife,Serv. De Comerc. Y Ases. Callbusiness SPA,340016490,EMP6008,A-0173
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,EMP6274,A-0353
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,EMP6274,A-0353
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,EMP6274,A-0353
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,EMP6274,A-0353


In [34]:
#pacientes_empresas22 = pd.merge(
    #left=Pacientes_SF,
    #right=empresas_unidas,
    #how="left",
    #left_on="Id Account Paciente",
    #right_on="Id Account", 
#)
#pacientes_empresas22 = pacientes_empresas22.drop_duplicates(['Rut'], keep='last')
#pacientes_empresas22 = pacientes_empresas22[
    #["Rut", "Nombre", "Nombre Campaña", "Nombre Empresa"]
#]


In [35]:
#pacientes_empresas22.shape

In [36]:
#apprix_1 = pacientes_empresas22.iloc[:848826,:]

#apprix_1.to_excel("F:/Empresas SF/reportes/Ruts_habilitados1.xlsx",index=False)


In [37]:
#apprix_2 = pacientes_empresas22.iloc[848827:,:]#
#apprix_2.to_excel("F:/Empresas SF/reportes/Ruts_habilitados2.xlsx",index=False)

In [38]:
#pacientes_empresas22.head()

Carga de Archivo carga

In [24]:
nuevos = "F:/Empresas SF/reportes/carga.xlsx"
campaña = "THB Seguros COVID"
Pais = "Chile"
#Pais = "México"

Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion


In [25]:
Pacientes_Archivo.head(10)

,Nombre de Empleador,Rut de empleador,Rut de trabajador,DV,rut1,Nombre de trabajador,apellidos,nombres,Inicio de vigencia
0,CONSTRUCTORA DON PEDRO LTDA,76144575-8,18107426,4,18107426-4,BARRA CHAVEZ CHRISTOPHER ELIAS,BARRA CHAVEZ,CHRISTOPHER ELIAS,2023-03-02
1,CONSTRUCTORA DON PEDRO LTDA,76144575-8,20811523,5,20811523-5,CARTES SAAVEDRA RODRIGO IGNACIO,CARTES SAAVEDRA,RODRIGO IGNACIO,2023-03-02
2,CONSTRUCTORA DON PEDRO LTDA,76144575-8,16452219,9,16452219-9,QUINTUI VASQUEZ GERMAN DAVID,QUINTUI VASQUEZ,GERMAN DAVID,2023-03-02
3,CONSTRUCTORA DON PEDRO LTDA,76144575-8,20451717,7,20451717-7,AGUILERA FELLAY SEBASTIAN ALEXANDER,AGUILERA FELLAY,SEBASTIAN ALEXANDER,2023-03-02
4,CONSTRUCTORA DON PEDRO LTDA,76144575-8,9548786,6,9548786-6,CONTRERAS DIAZ MARIA EUGENIA,CONTRERAS DIAZ,MARIA EUGENIA,2023-03-02
5,CONSTRUCTORA DON PEDRO LTDA,76144575-8,14206746,3,14206746-3,CRUZ HEPULLAN LUIS ALBERTO,CRUZ HEPULLAN,LUIS ALBERTO,2023-03-02
6,CONSTRUCTORA DON PEDRO LTDA,76144575-8,20621569,0,20621569-0,MELO ERICES HECTOR GERARDO,MELO ERICES,HECTOR GERARDO,2023-03-02
7,CONSTRUCTORA DON PEDRO LTDA,76144575-8,21419818,5,21419818-5,MUÑOZ BARBERIS VICENTE JESUS,MUÑOZ BARBERIS,VICENTE JESUS,2023-03-02
8,CONSTRUCTORA DON PEDRO LTDA,76144575-8,9569610,4,9569610-4,ROCHA URIBE APOLONIO TOMAS,ROCHA URIBE,APOLONIO TOMAS,2023-03-02
9,CONSTRUCTORA DON PEDRO LTDA,76144575-8,13578503,2,13578503-2,SAN MARTIN CASTRO CECILIA DEL PILAR,SAN MARTIN,CASTRO CECILIA DEL PILAR,2023-03-02


In [26]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].astype(str)


In [27]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("-", "")

Pacientes_Archivo.head()

,Nombre de Empleador,Rut de empleador,Rut de trabajador,DV,rut1,Nombre de trabajador,apellidos,nombres,Inicio de vigencia,rut
0,CONSTRUCTORA DON PEDRO LTDA,76144575-8,18107426,4,18107426-4,BARRA CHAVEZ CHRISTOPHER ELIAS,BARRA CHAVEZ,CHRISTOPHER ELIAS,2023-03-02,181074264
1,CONSTRUCTORA DON PEDRO LTDA,76144575-8,20811523,5,20811523-5,CARTES SAAVEDRA RODRIGO IGNACIO,CARTES SAAVEDRA,RODRIGO IGNACIO,2023-03-02,208115235
2,CONSTRUCTORA DON PEDRO LTDA,76144575-8,16452219,9,16452219-9,QUINTUI VASQUEZ GERMAN DAVID,QUINTUI VASQUEZ,GERMAN DAVID,2023-03-02,164522199
3,CONSTRUCTORA DON PEDRO LTDA,76144575-8,20451717,7,20451717-7,AGUILERA FELLAY SEBASTIAN ALEXANDER,AGUILERA FELLAY,SEBASTIAN ALEXANDER,2023-03-02,204517177
4,CONSTRUCTORA DON PEDRO LTDA,76144575-8,9548786,6,9548786-6,CONTRERAS DIAZ MARIA EUGENIA,CONTRERAS DIAZ,MARIA EUGENIA,2023-03-02,95487866


In [28]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")

# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()

Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head() 

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_20224\485331713.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")


,Nombre de Empleador,Rut de empleador,Rut de trabajador,DV,rut1,Nombre de trabajador,apellidos,nombres,Inicio de vigencia,rut
0,CONSTRUCTORA DON PEDRO LTDA,76144575-8,18107426,4,18107426-4,BARRA CHAVEZ CHRISTOPHER ELIAS,BARRA CHAVEZ,CHRISTOPHER ELIAS,2023-03-02,181074264
1,CONSTRUCTORA DON PEDRO LTDA,76144575-8,20811523,5,20811523-5,CARTES SAAVEDRA RODRIGO IGNACIO,CARTES SAAVEDRA,RODRIGO IGNACIO,2023-03-02,208115235
2,CONSTRUCTORA DON PEDRO LTDA,76144575-8,16452219,9,16452219-9,QUINTUI VASQUEZ GERMAN DAVID,QUINTUI VASQUEZ,GERMAN DAVID,2023-03-02,164522199
3,CONSTRUCTORA DON PEDRO LTDA,76144575-8,20451717,7,20451717-7,AGUILERA FELLAY SEBASTIAN ALEXANDER,AGUILERA FELLAY,SEBASTIAN ALEXANDER,2023-03-02,204517177
4,CONSTRUCTORA DON PEDRO LTDA,76144575-8,9548786,6,9548786-6,CONTRERAS DIAZ MARIA EUGENIA,CONTRERAS DIAZ,MARIA EUGENIA,2023-03-02,95487866


In [29]:
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("-", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.strip()
#Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("-", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.strip()

Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.strip()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace(
    "-", ""
)


Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(
    Pacientes_SF["Rut"])

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_20224\2118328728.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_20224\2118328728.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")


In [30]:
#aca a los pacientes del archivo se les asigna sus datos en SF
Pacientes_con_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="Rut",
)
Pacientes_con_id.shape

(6980, 20)

In [31]:
Pacientes_con_id.head()

,Nombre de Empleador,Rut de empleador,Rut de trabajador,DV,rut1,Nombre de trabajador,apellidos,nombres,Inicio de vigencia,rut,RutEstaoNo,Id Account Paciente,Rut,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono
0,CONSTRUCTORA DON PEDRO LTDA,76144575-8,18107426,4,18107426-4,BARRA CHAVEZ CHRISTOPHER ELIAS,Barra Chavez,Christopher Elias,2023-03-02,181074264,True,0018c00002iww3eAAA,181074264,Christopher Elias Barra Chavez,RUT,None,None,None,None,None
1,CONSTRUCTORA DON PEDRO LTDA,76144575-8,20811523,5,20811523-5,CARTES SAAVEDRA RODRIGO IGNACIO,Cartes Saavedra,Rodrigo Ignacio,2023-03-02,208115235,True,0018c00002iww3fAAA,208115235,Rodrigo Ignacio Cartes Saavedra,RUT,None,None,None,None,None
2,CONSTRUCTORA DON PEDRO LTDA,76144575-8,16452219,9,16452219-9,QUINTUI VASQUEZ GERMAN DAVID,Quintui Vasquez,German David,2023-03-02,164522199,True,0018c00002iww3gAAA,164522199,German David Quintui Vasquez,RUT,None,None,None,None,None
3,CONSTRUCTORA DON PEDRO LTDA,76144575-8,20451717,7,20451717-7,AGUILERA FELLAY SEBASTIAN ALEXANDER,Aguilera Fellay,Sebastian Alexander,2023-03-02,204517177,True,0018c00002iww3hAAA,204517177,Sebastian Alexander Aguilera Fellay,RUT,None,None,None,None,None
4,CONSTRUCTORA DON PEDRO LTDA,76144575-8,9548786,6,9548786-6,CONTRERAS DIAZ MARIA EUGENIA,Contreras Diaz,Maria Eugenia,2023-03-02,95487866,True,0018c00002iww3iAAA,95487866,Maria Eugenia Contreras Diaz,RUT,None,None,None,None,None


In [32]:
# Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
Subir_pacientes["RutEstaoNo"] = Pacientes_con_id["RutEstaoNo"]
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]


if ("nombres" in Pacientes_con_id.columns) == True:
    Subir_pacientes["FirstName"] = Pacientes_con_id["nombres"]
if ("apellidos" in Pacientes_con_id.columns) == True:
    Subir_pacientes["LastName"] = Pacientes_con_id["apellidos"]
if ("rut" in Pacientes_con_id.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = Pacientes_con_id["rut"].astype(str).str.upper()
    Subir_pacientes.head(20)
if ("email" in Pacientes_con_id.columns) == True:
    # Pacientes_con_id["email"] = Pacientes_con_id["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = (
        Pacientes_con_id["email"].astype(str).str.lower().replace("nan", "")
    )
if ("nacimiento" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Pacientes_con_id["nacimiento"]

if ("sexo" in Pacientes_con_id.columns) == True:
    Sexo = {
        "M": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "O",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "n": "Prefiero no responder",
        "N": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
        "M ": "Hombre",
        "F ": "Mujer",
        'Varón':'Hombre'      
        
    }
    Pacientes_con_id.replace({"sexo": Sexo}, inplace=True)
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Pacientes_con_id["sexo"]
if ("direccion" in Pacientes_con_id.columns) == True:
    Subir_pacientes["BillingStreet"] = (
        Pacientes_con_id["direccion"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = (
        Pacientes_con_id["comuna"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = (
        Pacientes_con_id["ciudad"].str.title().replace("NaN", "")
    )

if ("cargo" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonTitle"] = Pacientes_con_id["cargo"].str.title()

if ("division" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonDepartment"] = Pacientes_con_id["division"].str.title()

if ("celular" in Pacientes_con_id.columns) == True:
    Subir_pacientes["Phone"] = Pacientes_con_id["celular"].apply(str).str.strip().replace("+","")

Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"




if Pais == 'Chile':        
    Subir_pacientes["BillingCountry"] = "Chile"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes['Pa_s_del_Paciente__c'] = 'a1U8c000005dAtnEAE'
else:    
    Subir_pacientes["BillingCountry"] = "México"
    Subir_pacientes["Identification_Type__c"] = "CURP"
    Subir_pacientes['Pa_s_del_Paciente__c'] = 'a1U8c000005dAtsEAE'


Subir_pacientes = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == False]


# Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account.xlsx", index=False)
Subir_pacientes.head()

,RutEstaoNo,FirstName,LastName,IdentificationId__pc,RecordTypeId,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c
3002,False,Carmen Gomez Zamora,Edenson Del,106330018,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE
3003,False,Silva Jerez,Romina Alicia,151192726,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE
3004,False,Martin Cesar Angel,Medina San,119127289,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE
3005,False,Mireya De Dios,Ramos Varela,119588901,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE
3006,False,Irene Del Carmen,Benitez Leiva,121012618,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE


In [33]:
clinica_virtual = pd.DataFrame()
clinica_virtual["rut"] = Pacientes_con_id["rut1"].str.upper().str.strip()
clinica_virtual["firstName"] = Pacientes_con_id["nombres"].str.title().str.strip()
clinica_virtual["lastName"] = Pacientes_con_id["apellidos"].str.title().str.strip()
clinica_virtual["accountId"] = campaña


Codigos_Clinica_Virtual = {
    "BCI": "1",
    "Farmacias Ahumada": "2",
    "MetLife": "3",
    "Vida Security": "4",
    "Willis": "5",
    "Entel": "6",
    "Testing": "7",
    "3SP": "8",
    "HomeCare": "9",
    "Pedro de Valdivia": "10",
    "Colaboradores": "11",
    "MineraAbra": "12",
    "CMPC": "13",
    "Lipigas": "14",
    "Asismed": "15",
    "Siempre Perú": "16",
    "Farmacias Cruz Verde": "17",
    "WFS Food Services": "18",
    "VidaCel": "19",
    "Dermik": "20",
    "EY": "21",
    "IND": "22",
    "Sindicato Walmart": "23",
    "Instruvalve": "24",
    "VidaCel Colaboradores": "25",
    "Socofar": "26",
    "Milan Fajanovic": "27",
    "THBseguros": "28",
    "Pesco": "29",
    "Burgos - Consorcio": "30",
    "COFAMA-SILENTIUM": "31",
    "KAUFMANN": "32",
    "QuePlan": "33",
    "Zurich": "34",
    "Colaboradores Siempre Chile": "35",
    "Talo": "36",
    "Transvip": "37",
    "Vida Security Colaboradores": "38",
    "FORUS": "39",
    "SURA": "40",
    "Southbrige": "41",
    "Construmart": "42",
    "BICE Vida": "43",
    "Lockton": "44",
    "Mercer": "45",
    "Grupo PSG": "46",
    "Siempre Chile": "47",
}

clinica_virtual.replace({"accountId": Codigos_Clinica_Virtual}, inplace=True)
clinica_virtual.to_excel("F:/Empresas SF/reportes/Clinica_Virtual.xlsx", index=False)

In [34]:
campaña_seleccionada = empresas_unidas[empresas_unidas["Campaña"] == campaña]
campaña_seleccionada.head()


,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Codigo EMP,Codigo A
160235,a1Y8c00000C6ViAEAV,0018c00002amVkIAAU,a1W8c0000083iddEAA,THB Seguros COVID,0018c00002cXJasAAG,Thb Seguros Covid Solo Titulares Pricewaterhou...,Solo Titulares,THB Seguros COVID,None,None,EMP6951,A-1151
160236,a1Y8c00000C6ViBEAV,0018c00002amVkJAAU,a1W8c0000083iddEAA,THB Seguros COVID,0018c00002cXJasAAG,Thb Seguros Covid Solo Titulares Pricewaterhou...,Solo Titulares,THB Seguros COVID,None,None,EMP6951,A-1151
160237,a1Y8c00000C6ViCEAV,0018c00002amVkKAAU,a1W8c0000083iddEAA,THB Seguros COVID,0018c00002cXJasAAG,Thb Seguros Covid Solo Titulares Pricewaterhou...,Solo Titulares,THB Seguros COVID,None,None,EMP6951,A-1151
160238,a1Y8c00000C6ViDEAV,0018c00002amVkMAAU,a1W8c0000083iddEAA,THB Seguros COVID,0018c00002cXJasAAG,Thb Seguros Covid Solo Titulares Pricewaterhou...,Solo Titulares,THB Seguros COVID,None,None,EMP6951,A-1151
160239,a1Y8c00000C6ViEEAV,0018c00002amVkUAAU,a1W8c0000083iddEAA,THB Seguros COVID,0018c00002cXJasAAG,Thb Seguros Covid Solo Titulares Pricewaterhou...,Solo Titulares,THB Seguros COVID,None,None,EMP6951,A-1151


In [35]:
campaña_seleccionada.shape

(199178, 12)

In [36]:
if campaña != "MetLife":
    pacientes_en_campaña = pd.merge(
        left=campaña_seleccionada,
        right=Pacientes_con_id,
        how="left",
        left_on="Id Account",
        right_on="Id Account Paciente",
    )
    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"
    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"
    pacientes_en_campaña.to_excel(
        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False
    )
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )

In [37]:
campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
campaña_en_pacientes = campaña_en_pacientes.rename(columns={'Programas_por_Empresas__c':'programas por empresa'})
campaña_en_pacientes['Paciente__c']=campaña_en_pacientes['Id Account Paciente']
campaña_en_pacientes['Programas_por_Empresas__c']=campaña_en_pacientes['programas por empresa']
campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )

In [38]:

#apprix_1 = Pacientes_SF.iloc[:790552,:]
#apprix_2 = Pacientes_SF.iloc[790553:,:]
#apprix_1.to_excel("F:/Empresas SF/reportes/Ruts1.xlsx",index=False)
#apprix_2.to_excel("F:/Empresas SF/reportes/Ruts2.xlsx",index=False)
